## Подредактируем dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def pereform(s: str):
  if len(s) != 0:
    s = s[1:-1]
    s = s.split(', ')
    res = []
    for i in s:
      res.append(i[1:-1].lower())
    return ' with ' + ', '.join(res)
  else:
    return ' with nothing'

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Lesson_9_Information_search/Lesson_9_Danila_Malinin/superheroes_nlp_dataset.csv')
df2 = df[['name', 'history_text', 'superpowers']]
for i in df2[df2['history_text'].isnull()].index:
     df2.iloc[i].history_text = 'nothing'
df2 = df2.dropna()
df2.superpowers = list(map(pereform, df2.superpowers))
df2['title'] = df2.name + df2.superpowers
res_df = df2[['title', 'history_text']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [5]:
res_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1448 entries, 0 to 1449
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1448 non-null   object
 1   history_text  1448 non-null   object
dtypes: object(2)
memory usage: 33.9+ KB


In [6]:
res_df.to_csv('dataframe_super.csv')

In [7]:
res_df.head()

,title,history_text
0,"3-D Man with super speed, super strength","Delroy Garrett, Jr. grew up to become a track ..."
1,"514A (Gotham) with durability, reflexes, super...",He was one of the many prisoners of Indian Hil...
2,"A-Bomb with accelerated healing, agility, bers...","Richard ""Rick"" Jones was orphaned at a young ..."
3,"Aa with energy absorption, energy armor, energ...",Aa is one of the more passive members of the P...
4,"Aaron Cash with weapon-based powers, weapons m...",Aaron Cash is the head of security at Arkham A...


In [8]:
df2 = res_df.copy()

## Нормализация текста

In [9]:
df2.to_csv('dataframe_1.csv', index=False)

### Обработка текста

In [10]:
import re

def get_clean(x):
  x = str(x).lower().replace('\\', '').replace('_', ' ')
  x = re.sub("(.)\\1{2,}", "\\1", x)
  x = re.sub(r'[^\w\s]','', x)
  return x

In [11]:
df2.history_text = [get_clean(i) for i in df2.history_text]
df2.title	 = [get_clean(i) for i in df2.title]

In [12]:
print(all(df2.applymap(lambda x: x is not None).history_text))
print(all(df2.applymap(lambda x: x is not None).title))

True
True


### Уменьшение словаря

In [13]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

sw_eng = set(stopwords.words('english'))
res1, res2 = [], []
for i in df2.history_text:
  res1.append(' '.join([word for word in i.split() if not word in sw_eng]))
df2.history_text = res1
for i in df2.title:
  res2.append(' '.join([word for word in i.split() if not word in sw_eng]))
df2.titlet = res2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':


### Нормализация

In [14]:
from nltk.stem.snowball import SnowballStemmer
from nltk import wordnet, pos_tag
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.wordnet.ADJ,
        'V': wordnet.wordnet.VERB,
        'N': wordnet.wordnet.NOUN,
        'R': wordnet.wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.wordnet.NOUN
stemmer = SnowballStemmer(language='english')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import WordNetLemmatizer
def my_lemmatizer(sent):
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                 for word, tag in pos_tag(tokenized_sent)]
    return ' '.join([lemmatizer.lemmatize(word, tag)
                    for word, tag in pos_tagged])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [15]:
from tqdm import tqdm
res1, res2 = [], []
for i in tqdm(df2.history_text):
  res1.append(my_lemmatizer(i))
for i in df2.title:
  res2.append(my_lemmatizer(i))
df2.history_text, df2.title = res1, res2

100%|██████████| 1448/1448 [00:55<00:00, 26.09it/s]


In [16]:
df2.head()

,title,history_text
0,3d man with super speed super strength,delroy garrett jr grow become track star compe...
1,514a gotham with durability reflex super strength,one many prisoner indian hill transfer another...
2,abomb with accelerated heal agility berserk mo...,richard rick jones orphan young age expel seve...
3,aa with energy absorption energy armor energy ...,aa one passive member pumice people race stone...
4,aaron cash with weaponbased power weapon master,aaron cash head security arkham asylum hook ha...


In [17]:
df2.to_csv('dataframe_2.csv', index=False)

In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1448 entries, 0 to 1449
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1448 non-null   object
 1   history_text  1448 non-null   object
dtypes: object(2)
memory usage: 33.9+ KB


In [19]:
q1 = np.array(df2.title)
q2 = np.array(df2.history_text)
df = pd.DataFrame()
df['title'] = q1
df['history_text'] = q2
df2 = df.copy()

In [20]:
id_title = {}
id_content = {}
for i in range(len(df2.title)):
  for j in df2.title[i].split():
    if j not in id_title:
       id_title[j] = set([i])
    else:
       id_title[j].add(i)
for i in tqdm(range(len(df2.history_text))):
  for j in df2.history_text[i].split():
    if j not in id_content:
       id_content[j] = set([i])
    else:
       id_content[j].add(i)

100%|██████████| 1448/1448 [00:00<00:00, 2538.40it/s]


In [21]:
import pickle

with open('id_title.pickle', 'wb') as f:
    pickle.dump(id_title, f)
with open('id_content.pickle', 'wb') as f:
    pickle.dump(id_content, f)

### Попытка создания что должно получиться:

У нас есть:
*   Не редактированный df
*   Нормализовайнный df
*   Словари в каких id встречаются те или иные слова

In [22]:
import pandas as pd
import numpy as np

In [23]:
df2 = pd.read_csv('/content/dataframe_2.csv')

In [24]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448 entries, 0 to 1447
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1448 non-null   object
 1   history_text  1448 non-null   object
dtypes: object(2)
memory usage: 22.8+ KB


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_1 = TfidfVectorizer()
vectorizer_1 = vectorizer_1.fit(df2.title)   # сделать просто fit
X = vectorizer_1.transform(df2.title)
zz = pd.DataFrame(X.toarray(), columns=vectorizer_1.get_feature_names_out())
zz.head()

,10,100,11,13,1610,1966,1997,2003,2006,2012,2099,3d,45,514a,52,65,aa,aaron,aayla,abbott,abe,ability,abin,abomb,abomination,abra,abraxas,absorb,absorption,acausality,accelerate,accelerated,accuser,ace,achilles,acidicus,acrobatics,adal,adam,adaptation,...,with,wiz,wizard,wolf,wolfsbane,wolverine,woman,wonder,wong,world,wotan,wraith,wu,wyatt,x23,xanadu,xman,xray,xs,yakushi,yamanaka,yang,yellow,yellowjacket,yoda,yondu,yoyo,yukio,zane,zatanna,zax,zemo,zero,zhang,zod,zola,zone,zoom,zurenarrh,ōtsutsuki
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.757642,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.099880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.703747,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.092776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.209645,0.0,0.0,0.0,0.0,0.075118,0.0,0.0,0.065667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.027638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.324922,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.116422,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.042835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.660353,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.087055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
vectorizer_2 = TfidfVectorizer()
vectorizer_2 = vectorizer_2.fit(df2.history_text)   # сделать просто fit
X = vectorizer_2.transform(df2.history_text)
zz2 = pd.DataFrame(X.toarray(), columns=vectorizer_2.get_feature_names_out())
zz2.head()

,002,003,004,0068,01,010,020,024,03,030,042,071,08,084,09,10,100,1000,100feet,100thfloor,100year,101,1014,103,105,107th,1090,10ft,10th,10yearold,11,1165,117,118,1191,11th,11yearold,12,120,1200,...,zsasz,zsaszs,zucco,zuccos,zuel,zuels,zuggernaut,zugir,zumac,zuras,zurenarrh,zurrenarrh,zx,zyx,älgrim,éowyn,ñoldor,ñoldors,ākādo,ďkidí,ōtsutsuki,ţepeş,ǣmortal,ˈnaɪəlɨs,звезда,крылова,ланья,петровна,сергеевна,тёмная,الساحر,الغول,تاليا,رأس,رعتكو,سراب,نيسا,アーカード,拳王,駄犬
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
zz.iloc[[0,1,2,3]]

,10,100,11,13,1610,1966,1997,2003,2006,2012,2099,3d,45,514a,52,65,aa,aaron,aayla,abbott,abe,ability,abin,abomb,abomination,abra,abraxas,absorb,absorption,acausality,accelerate,accelerated,accuser,ace,achilles,acidicus,acrobatics,adal,adam,adaptation,...,with,wiz,wizard,wolf,wolfsbane,wolverine,woman,wonder,wong,world,wotan,wraith,wu,wyatt,x23,xanadu,xman,xray,xs,yakushi,yamanaka,yang,yellow,yellowjacket,yoda,yondu,yoyo,yukio,zane,zatanna,zax,zemo,zero,zhang,zod,zola,zone,zoom,zurenarrh,ōtsutsuki
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.757642,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.099880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.703747,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.092776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.209645,0.0,0.0,0.0,0.0,0.075118,0.0,0.0,0.065667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.027638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.324922,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.116422,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.042835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
pip install joblib

In [8]:
import joblib
joblib.dump(vectorizer_1, 'vectroizer_1.pkl')
joblib.dump(vectorizer_2, 'vectroizer_2.pkl')

['vectroizer_2.pkl']

## Конвертируем в функции - результат

In [28]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import wordnet, pos_tag
from nltk import WordNetLemmatizer
nltk.download('stopwords')
sw_eng = set(stopwords.words('english'))
stemmer = SnowballStemmer(language='english')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
def get_clean(x):
  x = str(x).lower().replace('\\', '').replace('_', ' ')
  x = re.sub("(.)\\1{2,}", "\\1", x)
  x = re.sub(r'[^\w\s]','', x) 
  return x
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.wordnet.ADJ,
        'V': wordnet.wordnet.VERB,
        'N': wordnet.wordnet.NOUN,
        'R': wordnet.wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.wordnet.NOUN
def my_lemmatizer(sent):
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                 for word, tag in pos_tag(tokenized_sent)]
    return ' '.join([lemmatizer.lemmatize(word, tag)
                    for word, tag in pos_tagged])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
def preprocessing(s: str):
  s = get_clean(s)
  slave = []
  for word in s.split():
    if not word in sw_eng:
         slave.append(word)
  s = ' '.join(slave)
  s = my_lemmatizer(s)
  return s

In [30]:
preprocessing('healing')

'heal'

In [31]:
X = vectorizer_2.transform([preprocessing('people wtite stories')])
qqq = pd.DataFrame(X.toarray(), columns=vectorizer_2.get_feature_names_out())
qqq.head()

,002,003,004,0068,01,010,020,024,03,030,042,071,08,084,09,10,100,1000,100feet,100thfloor,100year,101,1014,103,105,107th,1090,10ft,10th,10yearold,11,1165,117,118,1191,11th,11yearold,12,120,1200,...,zsasz,zsaszs,zucco,zuccos,zuel,zuels,zuggernaut,zugir,zumac,zuras,zurenarrh,zurrenarrh,zx,zyx,älgrim,éowyn,ñoldor,ñoldors,ākādo,ďkidí,ōtsutsuki,ţepeş,ǣmortal,ˈnaɪəlɨs,звезда,крылова,ланья,петровна,сергеевна,тёмная,الساحر,الغول,تاليا,رأس,رعتكو,سراب,نيسا,アーカード,拳王,駄犬
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
X = vectorizer_1.transform([preprocessing('people wtite stories')])
qq1 = pd.DataFrame(X.toarray(), columns=vectorizer_1.get_feature_names_out())
#qq1.drop(qq1.columns[list(range(384000, len(qq1.columns)))], axis = 1, inplace = True)
qq1.head()

,10,100,11,13,1610,1966,1997,2003,2006,2012,2099,3d,45,514a,52,65,aa,aaron,aayla,abbott,abe,ability,abin,abomb,abomination,abra,abraxas,absorb,absorption,acausality,accelerate,accelerated,accuser,ace,achilles,acidicus,acrobatics,adal,adam,adaptation,...,with,wiz,wizard,wolf,wolfsbane,wolverine,woman,wonder,wong,world,wotan,wraith,wu,wyatt,x23,xanadu,xman,xray,xs,yakushi,yamanaka,yang,yellow,yellowjacket,yoda,yondu,yoyo,yukio,zane,zatanna,zax,zemo,zero,zhang,zod,zola,zone,zoom,zurenarrh,ōtsutsuki
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
input_tf_idf = qqq.copy()

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
import multiprocessing as mp

input_tf_idf = np.array(input_tf_idf)[0]  # tf-idf нормализованной вводиомой строки 
res = []
for i in range(len(zz2)):
  qq = zz2.loc[i]
  #train_dense_masked = pd.concat([qqq, qq], ignore_index=True)  # долгоооо
  user_similarities = cosine_similarity([input_tf_idf, np.array(qq)])
  res.append((i, user_similarities[0, 1]))

In [35]:
sorted(res, key=lambda x: x[1], reverse=True)[0]

(1369, 0.2650844873452087)

↓ Даем входную строку, получаем индексы + coef релевантности документов

In [51]:
def find_super_charecters(st: str):
  k_title, k_content = 1.5, 0.5
  X = vectorizer_2.transform([preprocessing(st)])
  qqq = pd.DataFrame(X.toarray(), columns=vectorizer_2.get_feature_names_out()) # content tf-idf
  X = vectorizer_1.transform([preprocessing(st)])
  qq1 = pd.DataFrame(X.toarray(), columns=vectorizer_1.get_feature_names_out()) # title tf-idf
  content_tf_idf = np.array(qqq)[0].copy()
  title_tf_idf = np.array(qq1)[0]
  res = []
  for i in range(len(zz2)):
    dd1 = zz2.loc[i] # content
    dd2 = zz.loc[i]
    user_similarities1 = cosine_similarity([content_tf_idf, np.array(dd1)])[0, 1]
    user_similarities2 = cosine_similarity([title_tf_idf, np.array(dd2)])[0, 1]  # title
    res.append((i, user_similarities2 * k_title + user_similarities1 * k_content))
  return sorted(res, key=lambda x: x[1], reverse=True)[:30]

## Метрики качества

### precision@k

Как я понял, нужно разметить несклолько данный 0, 1 как кластеры и проверить качество мое модели = кол-во прав/кол-во всех правильных, как precision

In [57]:
def precision_k(real_id, model_id, k):
  model_id = model_id[:k]
  return sum([1 for i in model_id if i[0] in real_id])/len(model_id)

Для себя я выбрал что эти id то что я ищу

In [54]:
id_find = [1417, 640, 1418, 1419, 516, 1415]
model_predict = find_super_charecters('super women')
print(model_predict)

[(1417, 0.7447586679296212), (640, 0.6320709317704458), (1418, 0.5278192488362552), (1419, 0.40100724153503836), (263, 0.1559236221799768), (516, 0.15308690928874552), (1350, 0.14888980017761544), (1317, 0.14609082355120978), (1415, 0.13754349561974766), (720, 0.13491753021770211), (1071, 0.13449240451738806), (158, 0.13402661458669632), (285, 0.13402640384900358), (923, 0.1306487872151045), (1312, 0.12979067197294697), (408, 0.12789215789421687), (986, 0.1273899881934094), (430, 0.12606442846527252), (1314, 0.12177469666885758), (861, 0.12049552747941986), (1115, 0.11905834147941492), (32, 0.118417171554374), (1164, 0.11800762332718968), (828, 0.11800707728553642), (639, 0.11769676425343761), (15, 0.11699898690319219), (918, 0.11638296065930745), (62, 0.11624498089747522), (695, 0.11568185748712986), (1315, 0.11566750488559116)]


Посчитаем precision@k:

In [58]:
precision_k(id_find, model_predict, 10)

0.6

### MRR

In [71]:
def MRR_k(real_id, model_id, k):
  real_id = real_id[:k]
  halp_list = [i[0] for i in model_id]
  return sum([i/(halp_list.index(real_id[i])+1) for i in range(len(real_id)) if real_id[i] in halp_list])/len(real_id)

In [82]:
round(MRR_k(id_find, model_predict, 6), 3)

0.523

### DCG

In [84]:
def MRR_k(model_id, k):
  model_id = model_id[:k]
  return sum([model_id[i][1]/np.log(1 + i+1) for i in range(len(model_id))])

In [90]:
MRR_k(model_predict, 6)

2.4453900469481766